# Projet de Rendu 3D avec Calcul d'Éclairement

Ce projet implémente un moteur de rendu 3D simple en Python, capable de calculer l'éclairement d'objets maillés avec gestion des ombres portées et tessellation.

## Objectifs du TD

Le projet se décompose en 4 travaux dirigés (TD) progressifs qui permettent de construire un système complet de visualisation et d'éclairage d'objets 3D :

1. **TD1** : Manipulation de triangles et d'objets polyédriques
2. **TD2** : Calcul d'éclairement direct (sans ombres)
3. **TD3** : Calcul d'ombres portées par intersection rayon-triangle
4. **TD4** : Tessellation adaptative et rendu final

## Structure des données

Tous les objets 3D sont représentés par deux matrices numpy :
- **sommets** : matrice (N × 3) où chaque ligne contient les coordonnées (x, y, z) d'un sommet
- **triangles** : matrice (M × 3) où chaque ligne contient les indices de 3 sommets formant un triangle

Cette représentation est compatible avec le format retourné par `trimesh`.

## Installation des bibliothèques

Avant de commencer, assurez-vous d'avoir installé les dépendances nécessaires :

In [ ]:
# Installation des dépendances (décommentez si nécessaire)
# !pip install numpy matplotlib trimesh tqdm

## Imports nécessaires

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import trimesh
from tqdm import tqdm

# Configuration pour afficher les figures dans le notebook
%matplotlib inline

## Introduction à NumPy pour la 3D

Dans ce TD, nous allons utiliser **NumPy** pour manipuler des objets géométriques en 3D. Voici les concepts essentiels à connaître :

### 1. Définir un point 3D

Un point dans l'espace est représenté par un vecteur [x, y, z] sous forme de tableau NumPy :

```python
point = np.array([x, y, z])
```

### 2. Définir une matrice de sommets

Un ensemble de N sommets est représenté par une matrice (N × 3) où chaque ligne est un sommet :

```python
sommets = np.array([
    [x1, y1, z1],
    [x2, y2, z2],
    [x3, y3, z3],
    ...
])
```

### 3. Définir une matrice de triangles

Un ensemble de M triangles est représenté par une matrice (M × 3) d'**indices** :

```python
triangles = np.array([
    [i1, j1, k1],  # Triangle 1 : relie les sommets d'indices i1, j1, k1
    [i2, j2, k2],  # Triangle 2 : relie les sommets d'indices i2, j2, k2
    ...
])
```

### 4. Fonctions NumPy essentielles

Voici les fonctions que nous utiliserons fréquemment :

| Fonction | Description | Exemple |
|----------|-------------|---------|
| `np.array()` | Créer un tableau NumPy | `A = np.array([1, 2, 3])` |
| `np.dot(a, b)` | Produit scalaire | `np.dot([1,0,0], [0,1,0]) → 0` |
| `np.cross(a, b)` | Produit vectoriel | `np.cross([1,0,0], [0,1,0]) → [0,0,1]` |
| `np.linalg.norm(v)` | Norme (longueur) d'un vecteur | `np.linalg.norm([3,4,0]) → 5.0` |
| `np.max(M, axis=0)` | Maximum par colonne | Obtenir pmax d'un objet |
| `np.min(M, axis=0)` | Minimum par colonne | Obtenir pmin d'un objet |
| `np.vstack([M1, M2])` | Empiler verticalement | Fusionner des matrices |
| `np.delete(M, i, axis=0)` | Supprimer une ligne | Retirer un triangle |
| `np.zeros(n)` | Vecteur de zéros | Initialiser un vecteur |

### 5. Opérations vectorielles de base

```python
# Opérations élémentaires
A + B          # Addition de vecteurs
A - B          # Soustraction de vecteurs
k * A          # Multiplication par un scalaire
A / 2          # Division par un scalaire

# Opérations géométriques
np.dot(A, B)           # Produit scalaire : A · B
np.cross(A, B)         # Produit vectoriel : A × B
np.linalg.norm(A)      # Norme (longueur) : ||A||
A / np.linalg.norm(A)  # Normalisation : vecteur unitaire
```

Voyons maintenant quelques exemples concrets :

## Fonctions utilitaires d'affichage

Voici deux fonctions de base que nous utiliserons tout au long du TD pour créer et configurer nos figures 3D.

In [ ]:
def set_axes_equal(ax):
    """
    Définit un rapport d'aspect égal pour les axes 3D.
    Cela permet d'avoir la même échelle sur les 3 axes.
    
    Paramètres:
    -----------
    ax : matplotlib 3D Axes
        Les axes à configurer
    """
    x_min, x_max = ax.get_xlim()
    y_min, y_max = ax.get_ylim()
    z_min, z_max = ax.get_zlim()
    
    x_range = x_max - x_min
    y_range = y_max - y_min
    z_range = z_max - z_min
    
    max_range = max(x_range, y_range, z_range)
    
    x_center = (x_max + x_min) / 2
    y_center = (y_max + y_min) / 2
    z_center = (z_max + z_min) / 2
    
    ax.set_xlim(x_center - max_range/2, x_center + max_range/2)
    ax.set_ylim(y_center - max_range/2, y_center + max_range/2)
    ax.set_zlim(z_center - max_range/2, z_center + max_range/2)
    ax.set_box_aspect([1, 1, 1])


def figure():
    """
    Crée et configure une nouvelle figure 3D.
    
    Retourne:
    --------
    fig : matplotlib Figure
        La figure créée
    ax : matplotlib 3D Axes
        Les axes 3D configurés
    """
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.grid(True)
    return fig, ax

---
# TD1 : Manipulation de triangles et objets 3D

## Objectifs
- Manipuler des triangles individuels
- Créer et afficher des objets polyédriques simples
- Comprendre la représentation sommets/triangles

## PARTIE 1 : Triangle simple

Un triangle est défini par 3 sommets (x,y,z) stockés dans 3 vecteurs [1x3]

### Exercices :

**1. Fonction `calcul_centre_de_gravite(A, B, C)`**

Coder une fonction qui calcule et retourne le centre de gravité d'un triangle.

In [ ]:
def calcul_centre_de_gravite(A, B, C):
    """
    Calcule le centre de gravité d'un triangle.
    
    Paramètres:
    -----------
    A, B, C : array-like, shape (3,)
        Coordonnées des trois sommets (x, y, z)
    
    Retourne:
    --------
    G : ndarray, shape (3,)
        Coordonnées du centre de gravité
    """
    return (A + B + C) / 3

**2. Fonction `calcul_normale_triangle(A, B, C)`**

Coder une fonction qui calcule et retourne la normale unitaire (de norme 1) d'un triangle.

In [ ]:
def calcul_normale_triangle(A, B, C):
    """
    Calcule la normale unitaire d'un triangle.
    
    Paramètres:
    -----------
    A, B, C : array-like, shape (3,)
        Coordonnées des trois sommets (x, y, z)
    
    Retourne:
    --------
    normale : ndarray, shape (3,)
        Vecteur normal unitaire
    """
    AB = B - A
    AC = C - A
    normale = np.cross(AB, AC)
    return normale / np.linalg.norm(normale)

**3. Fonction `affichage_triangle(A, B, C, ax)`**

Coder une fonction qui affiche un triangle et sa normale (partant de son centre de gravité) dans un repère (ax) Matplotlib.

**Indications** :
- Utiliser `ax.plot()` pour dessiner les arêtes du triangle
- Utiliser `ax.quiver()` pour dessiner la normale comme une flèche
- Mettre à l'échelle la normale à ~30% de la longueur d'une arête pour qu'elle soit visible

In [ ]:
def affichage_triangle(A, B, C, ax=None):
    """
    Affiche un triangle en 3D avec son vecteur normal.
    
    Paramètres:
    -----------
    A, B, C : array-like, shape (3,)
        Coordonnées des trois sommets
    ax : matplotlib 3D Axes, optional
        Les axes sur lesquels dessiner. Si None, utilise les axes courants.
    """
    if ax is None:
        ax = plt.gca()
    
    # Dessiner les arêtes du triangle
    triangle = np.array([A, B, C, A])
    ax.plot(triangle[:, 0], triangle[:, 1], triangle[:, 2], 'blue')
    
    # Calculer et dessiner la normale
    G = calcul_centre_de_gravite(A, B, C)
    H = calcul_normale_triangle(A, B, C)
    
    # Mettre à l'échelle la normale à 30% de la longueur d'une arête
    H = H * np.linalg.norm(A - B) * 0.3
    
    ax.quiver(G[0], G[1], G[2], H[0], H[1], H[2], color='red', arrow_length_ratio=0.1)

**4. TEST**

Définir 3 points, créer une figure avec `figure()`, appeler `affichage_triangle()`

In [ ]:
# TEST - Triangle simple
p1 = np.array([0, 0, 0])
p2 = np.array([2, 0, 0])
p3 = np.array([1, 1, 3])

fig, ax = figure()
affichage_triangle(p1, p2, p3, ax)
set_axes_equal(ax)
plt.title("TD1 - Partie 1 : Triangle simple avec normale")
plt.show()

## PARTIE 2 : Pyramide à base carrée

Un objet 3D est représenté par deux matrices :
- **sommets** : matrice (N × 3) où chaque ligne contient les coordonnées (x, y, z) d'un sommet
- **triangles** : matrice (M × 3) où chaque ligne contient les **indices** de 3 sommets formant un triangle

### Exercices :

**1. Fonction `recupere_sommets(ind, sommets, triangles)`**

Coder une fonction qui extrait les coordonnées des 3 sommets d'un triangle à partir de son indice.

Cette fonction est très utile car elle permet de convertir les indices (dans la matrice `triangles`) en coordonnées réelles (depuis la matrice `sommets`).

In [ ]:
def recupere_sommets(ind, sommets, triangles):
    """
    Extrait les coordonnées des trois sommets d'un triangle.
    
    Paramètres:
    -----------
    ind : int
        Indice du triangle dans la matrice triangles
    sommets : ndarray, shape (N, 3)
        Matrice des coordonnées des sommets
    triangles : ndarray, shape (M, 3)
        Matrice des indices des triangles
    
    Retourne:
    --------
    A, B, C : ndarray, shape (3,)
        Coordonnées des trois sommets
    """
    i_A, i_B, i_C = triangles[ind]
    A = sommets[i_A]
    B = sommets[i_B]
    C = sommets[i_C]
    return A, B, C

**2. Fonction `creer_pyramide(base, hauteur)`**

Coder une fonction qui crée une pyramide à base carrée centrée à l'origine.

**Indications** :
- La pyramide a 5 sommets : 4 pour la base carrée + 1 pour le sommet
- Elle est composée de 6 triangles : 2 pour la base + 4 pour les côtés
- La base est dans le plan z=0
- Le sommet est au centre de la base, à hauteur z=hauteur

In [ ]:
def creer_pyramide(base, hauteur):
    """
    Crée une pyramide à base carrée centrée à l'origine.
    
    Paramètres:
    -----------
    base : float
        Longueur du côté de la base carrée
    hauteur : float
        Hauteur de la pyramide (selon l'axe z)
    
    Retourne:
    --------
    sommets : ndarray, shape (5, 3)
        Coordonnées des 5 sommets
    triangles : ndarray, shape (6, 3)
        Indices des triangles (2 pour la base, 4 pour les côtés)
    """
    b = base
    h = hauteur
    
    sommets = np.array([
        [0, 0, 0],
        [b, 0, 0],
        [b, b, 0],
        [0, b, 0],
        [b/2, b/2, h]
    ])
    
    triangles = np.array([
        [0, 2, 1],
        [0, 3, 2],
        [4, 0, 1],
        [4, 1, 2],
        [4, 2, 3],
        [4, 3, 0]
    ], dtype=int)
    
    return sommets, triangles

**3. Fonction `affichage_objet(sommets, triangles, ax)`**

Coder une fonction qui affiche un objet complet avec les normales de tous ses triangles.

**Indications** :
- Parcourir tous les triangles
- Pour chaque triangle, utiliser `recupere_sommets()` pour obtenir ses coordonnées
- Puis utiliser `affichage_triangle()` pour l'afficher

In [ ]:
def affichage_objet(sommets, triangles, ax=None):
    """
    Affiche un objet polyédrique avec les normales de tous les triangles.
    
    Paramètres:
    -----------
    sommets : ndarray, shape (N, 3)
        Coordonnées des sommets
    triangles : ndarray, shape (M, 3)
        Indices des triangles
    ax : matplotlib 3D Axes, optional
        Les axes sur lesquels dessiner. Si None, utilise les axes courants.
    """
    if ax is None:
        ax = plt.gca()
    
    nb_tri = len(triangles)
    for i in range(nb_tri):
        A, B, C = recupere_sommets(i, sommets, triangles)
        affichage_triangle(A, B, C, ax)
    set_axes_equal(ax)

**4. TEST**

Créer une pyramide avec `creer_pyramide()`, créer une figure avec `figure()`, afficher la pyramide avec `affichage_objet()`

In [ ]:
# TEST - Pyramide à base carrée
base = 6
height = 8
s, t = creer_pyramide(base, height)

fig, ax = figure()
affichage_objet(s, t, ax)
plt.title("TD1 - Partie 2 : Pyramide à base carrée avec normales")
plt.show()

---
# TD2 : Calcul et affichage de l'éclairement des triangles

## Objectifs
- Calculer l'éclairement direct d'une surface
- Visualiser l'éclairement avec une colormap
- Travailler avec des modèles 3D complexes (fichiers STL)

## Modèle d'éclairement

Nous considèrerons que l'éclairement d'un triangle dépend uniquement de l'angle entre :
- La **normale** au triangle
- Le **vecteur** allant du centre du triangle vers la source lumineuse

**Formule** : `E = max(0, cos(θ))` où θ est l'angle entre la normale et le vecteur vers la source.

- Si la normale "regarde" vers la source (θ < 90°), cos(θ) > 0 → le triangle est éclairé
- Si la normale "regarde" ailleurs (θ > 90°), cos(θ) < 0 → le triangle n'est pas éclairé (E = 0)

## PARTIE 1 : Éclairement sans ombres

### Exercices :

**1. Fonction `calcul_eclairement_triangle(A, B, C, source)`**

Coder une fonction qui calcule l'éclairement normalisé (entre 0 et 1) d'un triangle par une source luminuse ponctuelle.

In [ ]:
def calcul_eclairement_triangle(A, B, C, source):
    """
    Calcule l'éclairement direct normalisé d'un triangle.
    
    Paramètres:
    -----------
    A, B, C : array-like, shape (3,)
        Coordonnées des trois sommets
    source : array-like, shape (3,)
        Coordonnées de la source lumineuse
    
    Retourne:
    --------
    E : float
        Valeur d'éclairement normalisée (0 à 1)
        E = max(0, cos(θ)) où θ est l'angle entre la normale et la direction de la lumière
    """
    # Calculer le centre du triangle
    G = calcul_centre_de_gravite(A, B, C)
    
    # Vecteur du centre vers la source
    vecteur_rayon = source - G
    d = np.linalg.norm(vecteur_rayon)
    
    # Normale du triangle (vecteur unitaire)
    normale = calcul_normale_triangle(A, B, C)
    
    # Cosinus de l'angle entre la normale et le rayon lumineux
    costheta = np.dot(vecteur_rayon, normale) / d
    
    # Éclairement normalisé (pas de valeurs négatives)
    E = max(0, costheta)
    
    return E

**2. Fonction `calcul_eclairement_objet_simple(sommets, triangles, source)`**

Coder une fonction qui calcule l'éclairement de tous les triangles d'un objet (SANS tenir compte des ombres portées).

Elle doit retourner un vecteur contenant l'éclairement de chaque triangle.

In [ ]:
def calcul_eclairement_objet_simple(sommets, triangles, source):
    """
    Calcule l'éclairement direct de tous les triangles SANS ombres.
    
    Paramètres:
    -----------
    sommets : ndarray, shape (N, 3)
        Coordonnées des sommets
    triangles : ndarray, shape (M, 3)
        Indices des triangles
    source : array-like, shape (3,)
        Coordonnées de la source lumineuse
    
    Retourne:
    --------
    E : ndarray, shape (M,)
        Valeurs d'éclairement normalisées pour chaque triangle (0 à 1)
    """
    nb_tri = len(triangles)
    E = np.zeros(nb_tri)
    
    for i in range(nb_tri):
        A, B, C = recupere_sommets(i, sommets, triangles)
        E[i] = calcul_eclairement_triangle(A, B, C, source)
    
    return E

**3. Fonction `affichage_objet_eclaire(sommets, triangles, couleurs, source, ax)`**

Coder une fonction qui affiche un objet coloré selon son éclairement.

**Indications** :
- Utiliser `Poly3DCollection` pour créer une collection de triangles
- Utiliser `plt.cm.hot(couleurs)` pour convertir les valeurs d'éclairement en couleurs (colormap "hot")
- Afficher la source lumineuse avec `ax.scatter()`

In [ ]:
def affichage_objet_eclaire(sommets, triangles, couleurs, source, ax=None):
    """
    Affiche un objet polyédrique avec éclairage coloré et source lumineuse.
    
    Paramètres:
    -----------
    sommets : ndarray, shape (N, 3)
        Coordonnées des sommets
    triangles : ndarray, shape (M, 3)
        Indices des triangles
    couleurs : ndarray, shape (M,)
        Valeurs d'éclairement normalisées pour chaque triangle (0 à 1)
    source : array-like, shape (3,)
        Coordonnées de la source lumineuse
    ax : matplotlib 3D Axes, optional
        Les axes sur lesquels dessiner. Si None, utilise les axes courants.
    """
    if ax is None:
        ax = plt.gca()
    
    # Créer une collection de triangles avec couleurs
    verts = [[sommets[t[0]], sommets[t[1]], sommets[t[2]]] for t in triangles]
    collection = Poly3DCollection(verts, facecolors=plt.cm.hot(couleurs),
                                   edgecolors='k', linewidths=0.1, alpha=0.9)
    ax.add_collection3d(collection)
    
    # Afficher la source lumineuse
    ax.scatter(*source, color='yellow', s=200, marker='*', edgecolors='black', linewidths=2)
    
    # Définir les limites des axes avec échelles égales
    set_axes_equal(ax)

**4. TEST sur la pyramide**

Créer une pyramide, définir une position de source lumineuse, calculer le vecteur E d'éclairement, créer une figure, afficher le résultat.

In [ ]:
# TEST - Pyramide éclairée
base = 6
height = 8
s, t = creer_pyramide(base, height)
source = np.array([-10, 6, 10])

E = calcul_eclairement_objet_simple(s, t, source)

fig, ax = figure()
affichage_objet_eclaire(s, t, E, source, ax)
plt.title("TD2 - Partie 1 : Pyramide éclairée (sans ombres)")
plt.show()

## PARTIE 2 : Test sur un objet STL

### Exercices :

1. Utiliser `trimesh.load()` pour charger le fichier 'building.stl'
   - Le résultat contient directement `.vertices` (sommets) et `.faces` (triangles)

2. Positionner une source lumineuse en fonction de la taille du bâtiment
   - Astuce : utiliser `np.max(sommets, axis=0)` pour obtenir les coordonnées maximales
   - Placer la source à une distance proportionnelle (par exemple 3× la dimension maximale)

3. Calculer l'éclairement avec `calcul_eclairement_objet_simple()`

4. Afficher le bâtiment éclairé avec `affichage_objet_eclaire()`

In [ ]:
# TEST - Bâtiment éclairé (sans ombres)
# Note: Assurez-vous que le fichier building.stl est dans le même dossier

stl_mesh = trimesh.load('building.stl')
sommets = stl_mesh.vertices
triangles = stl_mesh.faces

# Positionner la source en fonction de la taille du bâtiment
pmax = np.max(sommets, axis=0)
source = pmax * 3
source[1] = -source[1]  # Inverser Y pour placer la source devant

E = calcul_eclairement_objet_simple(sommets, triangles, source)

fig, ax = figure()
affichage_objet_eclaire(sommets, triangles, E, source, ax)
plt.title("TD2 - Partie 2 : Bâtiment éclairé (sans ombres)")
plt.show()

## PARTIE 3 : Bonus (optionnel)

Afficher également le bâtiment avec `affichage_objet()` pour visualiser toutes les normales des triangles.

**Attention** : Peut être lent avec beaucoup de triangles !

In [ ]:
# BONUS - Affichage des normales du bâtiment (décommenter pour exécuter)
# Attention : peut être très lent avec beaucoup de triangles

fig, ax = figure()
affichage_objet(sommets, triangles, ax)
plt.title("TD2 - Bonus : Bâtiment avec toutes les normales")
plt.show()

---
# TD3 : Calcul des ombres par intersection rayon-triangle

## Objectifs
- Implémenter l'intersection rayon-triangle
- Détecter les ombres portées
- Améliorer le réalisme du rendu

## Algorithme de détection des ombres

Pour chaque triangle éclairé :
1. Tracer un rayon entre le **centre du triangle** et la **source lumineuse**
2. Tester si ce rayon **intersecte un autre triangle**
3. Si oui, le triangle est **à l'ombre** : éclairement = 0

## PARTIE 1 : Tests d'intersection segment/triangle

### Exercices :

**1. Fonction `calcul_intersection_triangle_segment(A, B, C, P1, P2)`**

Coder une fonction qui calcule l'intersection entre un triangle (A, B, C) et un segment [P1, P2] en vous aidant de  la description de l'algorithme dans le document `Calcul_Intersection_Triangle_Segment.pdf`

La fonction doit retourner :
- **test** : booléen indiquant si une intersection existe
- **I** : point d'intersection (ou None si pas d'intersection)

In [ ]:
def calcul_intersection_triangle_segment(A, B, C, P1, P2):
    """
    Calcule l'intersection entre un triangle et un segment dans l'espace 3D.
    
    Algorithme : Intersection rayon-plan suivie d'un test point-dans-triangle.
    
    Paramètres:
    -----------
    A, B, C : array-like, shape (3,)
        Sommets du triangle
    P1, P2 : array-like, shape (3,)
        Extrémités du segment
    
    Retourne:
    --------
    test : bool
        True si une intersection existe dans le segment [P1, P2]
    I : ndarray, shape (3,) or None
        Coordonnées du point d'intersection (None si pas d'intersection)
    """
    test = False
    I = None
    
    N = calcul_normale_triangle(A, B, C)
    direction = P2 - P1
    den = np.dot(N, direction)
    
    if abs(den) > np.finfo(float).eps:  # Segment non parallèle au plan du triangle
        t = np.dot(N, (A - P1)) / den
        
        if 0 <= t <= 1:  # Intersection dans les limites du segment
            I = P1 + t * direction
            
            # Vérifier si le point d'intersection est à l'intérieur du triangle
            # (le point est à l'intérieur s'il est "à gauche" de toutes les arêtes)
            if (np.dot(N, np.cross(B - A, I - A)) >= 0 and
                np.dot(N, np.cross(C - B, I - B)) >= 0 and
                np.dot(N, np.cross(A - C, I - C)) >= 0):
                test = True
    
    return test, I

**2. TEST**

Écrire un script qui teste le fonctionnement de la fonction `calcul_intersection_triangle_segment`.

Cas à tester :
- Segments qui intersectent le triangle (affichés en vert)
- Segments qui passent à côté (affichés en rouge)
- Segments qui passent par les sommets ou les arêtes
- Segments parallèles au plan du triangle

In [ ]:
# TEST - Intersection segment/triangle
A = np.array([0, 0, 0])
B = np.array([2, 0, 0])
C = np.array([1, 1, 3])

fig, ax = figure()
affichage_triangle(A, B, C, ax)

source = np.array([1, -6, 1])
points = []
y = 2
for x in np.arange(-1, 4, 1.8):
    for z in np.arange(-1, 4, 1.8):
        points.append([x, y, z])

# Ajouter des points spéciaux (milieux d'arêtes et sommets)
points.extend([
    (A + B) / 2,
    (B + C) / 2,
    (A + C) / 2,
    A,
    B,
    C
])

points = np.array(points)

# Tester l'intersection pour chaque point
for i in range(len(points)):
    test, I = calcul_intersection_triangle_segment(A, B, C, source, points[i])
    if test:
        # Intersection trouvée - ligne verte
        ax.plot([source[0], I[0]], [source[1], I[1]], [source[2], I[2]],
                'green', linewidth=2)
        ax.scatter(I[0], I[1], I[2], color='green', s=50)
    else:
        # Pas d'intersection - ligne rouge
        ax.plot([source[0], points[i, 0]], [source[1], points[i, 1]],
                [source[2], points[i, 2]], 'red', linewidth=1, alpha=0.5)

plt.title("TD3 - Partie 2 : Tests d'intersection rayon-triangle\n(Vert = intersection, Rouge = pas d'intersection)")
plt.show()

## PARTIE 2 : Calcul des éclairements avec ombres

### Exercices :

**1. Fonction `calcul_eclairement_objet(sommets, triangles, source)`**

Coder une fonction qui calcule l'éclairement de tous les triangles EN TENANT COMPTE des ombres portées.

**Note** : Ce calcul est en O(n²) et peut prendre plusieurs secondes. On utilise `tqdm` pour afficher une barre de progression.

In [ ]:
def calcul_eclairement_objet(sommets, triangles, source):
    """
    Calcule l'éclairement de tous les triangles AVEC calcul des ombres.
    
    Pour chaque triangle :
    1. Calculer l'éclairement de base selon l'angle avec la source
    2. Si face à la lumière, tester le rayon du centre du triangle vers la source
    3. Vérifier l'intersection avec tous les autres triangles
    4. Mettre l'éclairement à 0 si un triangle bloque la lumière
    
    Paramètres:
    -----------
    sommets : ndarray, shape (N, 3)
        Coordonnées des sommets
    triangles : ndarray, shape (M, 3)
        Indices des triangles
    source : array-like, shape (3,)
        Coordonnées de la source lumineuse
    
    Retourne:
    --------
    E : ndarray, shape (M,)
        Valeurs d'éclairement normalisées pour chaque triangle (0 à 1)
        Les triangles à l'ombre ont une valeur de 0
    """
    nb_tri = len(triangles)
    E = np.zeros(nb_tri)
    
    for i in tqdm(range(nb_tri), desc="Calcul de l'éclairement"):
        A, B, C = recupere_sommets(i, sommets, triangles)
        E[i] = calcul_eclairement_triangle(A, B, C, source)
        
        if E[i] > 0:  # Si le triangle fait face à la lumière
            G = calcul_centre_de_gravite(A, B, C)
            
            # Tester l'intersection avec tous les autres triangles
            for j in range(nb_tri):
                if i != j:  # Ne pas tester le triangle avec lui-même
                    a, b, c = recupere_sommets(j, sommets, triangles)
                    test, I = calcul_intersection_triangle_segment(a, b, c, G, source)
                    
                    if test:
                        # Intersection trouvée : le triangle est à l'ombre
                        E[i] = 0
                        break  # Pas besoin de tester les autres triangles
    
    return E

**2. TEST sur building.stl**

Tester ce nouveau calcul d'éclairement sur le bâtiment.

**Observation** : Vous devriez voir apparaître des ombres portées qui rendent le rendu beaucoup plus réaliste !

**Note** : Le calcul peut prendre quelques secondes (voire minutes) selon le nombre de triangles.

In [ ]:
# TEST - Bâtiment éclairé avec ombres
stl_mesh = trimesh.load('building.stl')

pmax = np.max(stl_mesh.vertices, axis=0)
source = pmax * 3
source[1] = -source[1]

print("Calcul de l'éclairement avec ombres portées...")
E = calcul_eclairement_objet(stl_mesh.vertices, stl_mesh.faces, source)

fig, ax = figure()
affichage_objet_eclaire(stl_mesh.vertices, stl_mesh.faces, E, source, ax)
plt.title("TD3 - Partie 3 : Bâtiment éclairé avec ombres portées")
plt.show()

---
# TD4 : Tessellation et raffinement de maillage

## Objectifs
- Raffiner le maillage pour améliorer la qualité du rendu
- Fusionner plusieurs objets (bâtiment + sol)
- Créer un rendu final de haute qualité

## Tessellation adaptative

La tessellation subdivise récursivement les triangles dont la surface dépasse un seuil :
- Chaque triangle est divisé en **4 sous-triangles** (en ajoutant 3 points au milieu des arêtes)
- Le processus continue jusqu'à ce que tous les triangles respectent le critère de surface

**Pourquoi ?** Un maillage plus fin permet un calcul d'éclairement plus précis et un rendu de meilleure qualité.

## PARTIE 1 : Tessellation d'un triangle

### Exercices :

**1. Fonction `calcul_surface_triangle(A, B, C)`**

Coder une fonction qui calcule l'aire d'un triangle.

**Rappel** : Aire = ||AB × AC|| / 2

In [ ]:
def calcul_surface_triangle(A, B, C):
    """
    Calcule l'aire d'un triangle.
    
    Paramètres:
    -----------
    A, B, C : array-like, shape (3,)
        Coordonnées des trois sommets (x, y, z)
    
    Retourne:
    --------
    surface : float
        Aire du triangle
    """
    AB = B - A
    AC = C - A
    return np.linalg.norm(np.cross(AB, AC)) * 0.5

**2. Fonction `tessellation_triangle(ind, sommets, triangles)`**

Coder une fonction qui subdivise un triangle en 4 sous-triangles de taille égale.

**Principe** :
1. Extraire les coordonnées A, B, C du triangle à l'indice `ind`
2. Calculer les 3 milieux des arêtes : M_AB, M_BC, M_CA
3. Créer 4 nouveaux triangles :
   - Triangle au coin A : [A, M_AB, M_CA]
   - Triangle au coin B : [M_AB, B, M_BC]
   - Triangle au coin C : [M_CA, M_BC, C]
   - Triangle central : [M_AB, M_BC, M_CA]

**Retours** :
- `nouveaux_sommets` : les 3 points au milieu des arêtes (seront ajoutés à `sommets`)
- `nouveaux_triangles` : les 4 triangles qui remplaceront l'ancien

In [ ]:
def tessellation_triangle(ind, sommets, triangles):
    """
    Subdivise un triangle en 4 sous-triangles en ajoutant des sommets au milieu des arêtes.
    
    Paramètres:
    -----------
    ind : int
        Indice du triangle à subdiviser
    sommets : ndarray, shape (N, 3)
        Coordonnées des sommets
    triangles : ndarray, shape (M, 3)
        Indices des triangles
    
    Retourne:
    --------
    nouveaux_sommets : ndarray, shape (3, 3)
        Les 3 nouveaux sommets au milieu des arêtes (seront ajoutés à sommets)
    nouveaux_triangles : ndarray, shape (4, 3)
        Les 4 triangles qui remplacent l'original
    """
    A, B, C = recupere_sommets(ind, sommets, triangles)
    
    # Créer de nouveaux sommets aux milieux des arêtes
    m_ab = (A + B) * 0.5
    m_bc = (B + C) * 0.5
    m_ca = (C + A) * 0.5
    
    nouveaux_sommets = np.array([m_ab, m_bc, m_ca])
    
    # Les nouveaux sommets seront ajoutés à la fin du tableau de sommets
    nb = len(sommets)
    i_ab = nb
    i_bc = nb + 1
    i_ca = nb + 2
    
    # Créer 4 nouveaux triangles pour remplacer l'original
    i_A, i_B, i_C = triangles[ind]
    
    nouveaux_triangles = np.array([
        [i_A, i_ab, i_ca],
        [i_ab, i_bc, i_ca],
        [i_ab, i_B, i_bc],
        [i_ca, i_bc, i_C]
    ], dtype=int)
    
    return nouveaux_sommets, nouveaux_triangles

**3. TEST**

Tester la tessellation sur un triangle simple.

In [ ]:
# TEST - Tessellation d'un triangle
A = np.array([0, 0, 0])
B = np.array([2, 0, 0])
C = np.array([1, 1, 3])

s1 = np.array([A, B, C])
t1 = np.array([[0, 1, 2]])

s2, t2 = tessellation_triangle(0, s1, t1)
s = np.vstack([s1, s2])
t = t2

fig, ax = figure()
affichage_objet(s, t, ax)
plt.title("TD4 - Partie 1 : Tessellation d'un triangle\n(1 triangle → 4 sous-triangles)")
plt.show()

## PARTIE 2 : Tessellation d'un objet complet

### Exercices :

**1. Fonction `tessellation_objet(sommets, triangles, surf_min)`**

Coder une fonction qui raffine un maillage en subdivisant récursivement tous les triangles dont la surface dépasse un seuil.

**Algorithme** :
```
i = 0
tant que i < nombre de triangles :
    calculer surface du triangle i
    si surface > surf_min :
        subdiviser le triangle i en 4 sous-triangles
        ajouter les 3 nouveaux sommets à la fin de sommets
        supprimer le triangle i de la liste
        ajouter les 4 nouveaux triangles à la fin
        # Ne pas incrémenter i (on re-teste le triangle à cette position)
    sinon :
        i = i + 1
```

**Important** : On n'incrémente `i` que si le triangle n'est PAS subdivisé. Cela permet de re-tester les nouveaux triangles.

In [ ]:
def tessellation_objet(sommets, triangles, surf_min):
    """
    Raffine un maillage en subdivisant récursivement les triangles dépassant un seuil de surface.
    
    Subdivise itérativement les triangles plus grands que surf_min :
    1. Parcourir tous les triangles (la liste grandit pendant l'itération)
    2. Si surface du triangle > surf_min, subdiviser en 4 triangles
    3. Supprimer le triangle original, ajouter les nouveaux sommets et triangles
    4. N'incrémenter l'indice que quand le triangle n'est PAS subdivisé
    
    Paramètres:
    -----------
    sommets : ndarray, shape (N, 3)
        Coordonnées des sommets
    triangles : ndarray, shape (M, 3)
        Indices des triangles
    surf_min : float
        Surface maximale autorisée pour un triangle
    
    Retourne:
    --------
    sommets_out : ndarray
        Coordonnées des sommets raffinés
    triangles_out : ndarray
        Indices des triangles raffinés
    """
    sommets_out = sommets.copy()
    triangles_out = triangles.copy()
    
    i = 0
    while i < len(triangles_out):
        A, B, C = recupere_sommets(i, sommets_out, triangles_out)
        s = calcul_surface_triangle(A, B, C)
        
        if s > surf_min:
            # Subdiviser ce triangle
            nouveaux_som, nouveaux_tri = tessellation_triangle(i, sommets_out, triangles_out)
            
            # Ajouter les nouveaux sommets
            sommets_out = np.vstack([sommets_out, nouveaux_som])
            
            # Supprimer l'ancien triangle et ajouter les nouveaux
            triangles_out = np.delete(triangles_out, i, axis=0)
            triangles_out = np.vstack([triangles_out, nouveaux_tri])
            
            # Ne pas incrémenter i - vérifier cette position à nouveau avec le nouveau triangle
        else:
            i += 1
    
    return sommets_out, triangles_out

**2. TEST**

Tester la tessellation d'un objet avec surf_min = 0.1

In [ ]:
# TEST - Tessellation d'un objet
s, t = tessellation_objet(s1, t1, 0.1)

fig, ax = figure()
affichage_objet(s, t, ax)
plt.title(f"TD4 - Partie 2 : Tessellation d'un objet\n(1 triangle → {len(t)} sous-triangles)")
plt.show()

## PARTIE 3 : Ajout d'un sol sous le bâtiment

### Exercices :

**1. Fonction `creer_sol(p1, p2)`**

Coder une fonction qui crée un plan rectangulaire horizontal défini par deux coins opposés.

Le sol est composé de 4 sommets et 2 triangles.

In [ ]:
def creer_sol(p1, p2):
    """
    Crée un plan de sol rectangulaire défini par deux coins opposés.
    
    Paramètres:
    -----------
    p1, p2 : array-like, shape (3,)
        Deux coins opposés du rectangle.
        La coordonnée z définit l'altitude du sol.
    
    Retourne:
    --------
    sommets : ndarray, shape (4, 3)
        Coordonnées des 4 coins
    triangles : ndarray, shape (2, 3)
        Indices des triangles formant le sol
    """
    p1, p2 = np.array(p1), np.array(p2)
    
    sommets = np.array([
        p1,
        [p2[0], p1[1], p1[2]],
        p2,
        [p1[0], p2[1], p1[2]]
    ])
    
    triangles = np.array([
        [0, 1, 2],
        [0, 2, 3]
    ], dtype=int)
    
    return sommets, triangles

**2. Fonction `creer_concatenation(s1, t1, s2, t2)`**

Coder une fonction qui fusionne deux objets maillés en un seul.

**Principe** :
1. Concaténer les matrices de sommets : `[s1; s2]`
2. Concaténer les matrices de triangles en **ajustant les indices** du second objet :
   - Les indices de `t2` pointent vers les sommets de `s2`
   - Après concaténation, les sommets de `s2` sont décalés de `len(s1)` positions
   - Donc il faut ajouter `len(s1)` à tous les indices de `t2`

In [ ]:
def creer_concatenation(s1, t1, s2, t2):
    """
    Fusionne deux maillages en un seul.
    
    Paramètres:
    -----------
    s1, t1 : ndarray
        Sommets et triangles du premier maillage
    s2, t2 : ndarray
        Sommets et triangles du second maillage
    
    Retourne:
    --------
    sommets : ndarray
        Sommets combinés
    triangles : ndarray
        Triangles combinés avec indices ajustés pour le second maillage
    """
    sommets = np.vstack([s1, s2])
    n = len(s1)
    triangles = np.vstack([t1, t2 + n])
    return sommets, triangles

**3. TEST**

Créer un sol sous le bâtiment :
1. Charger building.stl
2. Calculer les dimensions du bâtiment (pmin, pmax)
3. Créer un sol légèrement plus grand que le bâtiment
4. Fusionner le bâtiment et le sol avec `creer_concatenation()`
5. Calculer l'éclairement avec ombres
6. Afficher le résultat

In [ ]:
# TEST - Bâtiment avec sol
stl_mesh = trimesh.load('building.stl')

pmax = np.max(stl_mesh.vertices, axis=0)
source = pmax * 3
source[1] = -source[1]

# Calculer les dimensions pour créer un sol adapté
pmin = np.min(stl_mesh.vertices, axis=0)
h0 = pmin[2]  # Altitude du sol
delta = np.linalg.norm(pmax - pmin)

# Créer un sol légèrement plus grand que le bâtiment
pmin_sol = pmin - 1 * delta
pmax_sol = pmax + 1 * delta
pmax_sol[2] = h0
pmin_sol[2] = h0

s_sol, t_sol = creer_sol(pmin_sol, pmax_sol)
s, t = creer_concatenation(stl_mesh.vertices, stl_mesh.faces, s_sol, t_sol)

print("Calcul de l'éclairement avec ombres...")
E = calcul_eclairement_objet(s, t, source)

fig, ax = figure()
affichage_objet_eclaire(s, t, E, source, ax)
plt.title("TD4 - Partie 3 : Bâtiment avec sol et ombres portées")
plt.show()

## PARTIE 4 : Rendu final avec tessellation

**Objectif** : Obtenir un rendu d'éclairage de meilleure qualité en raffinant le maillage du bâtiment.

### Exercices :

1. **Calculer une surface minimale adaptative**
   - Trouver la surface du plus grand triangle du bâtiment (s_max)
   - Choisir surf_min = s_max / 5 (ou une autre fraction)
   - Cela permet d'avoir une tessellation adaptée à la taille de l'objet

2. **Appliquer la tessellation** sur l'ensemble bâtiment + sol

3. **Calculer l'éclairement avec ombres** sur le maillage raffiné

4. **Afficher le résultat final**

**Observation** : Le rendu d'éclairage devrait être plus précis grâce au maillage plus fin !

**Note** : Le calcul peut prendre plusieurs minutes selon la finesse de la tessellation.

In [ ]:
# TEST - Rendu final avec tessellation

# 1. Calculer la surface maximale du bâtiment
s_max = 0
for i in range(len(stl_mesh.faces)):
    A, B, C = recupere_sommets(i, stl_mesh.vertices, stl_mesh.faces)
    surf = calcul_surface_triangle(A, B, C)
    if surf > s_max:
        s_max = surf

# 2. Définir le seuil de tessellation
s_min = s_max / 5
print(f"Surface maximale : {s_max:.4f}")
print(f"Surface minimale pour tessellation : {s_min:.4f}")

# 3. Appliquer la tessellation
print(f"Nombre de triangles avant tessellation : {len(t)}")
s_final, t_final = tessellation_objet(s, t, s_min)
print(f"Nombre de triangles après tessellation : {len(t_final)}")

# 4. Calculer l'éclairement avec ombres
print("Calcul de l'éclairement avec ombres...")
E = calcul_eclairement_objet(s_final, t_final, source)

# 5. Afficher le résultat final
fig, ax = figure()
affichage_objet_eclaire(s_final, t_final, E, source, ax)
plt.title("TD4 - Partie 4 : Rendu final avec tessellation adaptative")
plt.show()

---
# Conclusion

**Félicitations !** Vous avez terminé les 4 TD de rendu 3D et créé un moteur de rendu complet avec gestion des ombres et tessellation adaptative.

## Récapitulatif des notions abordées

### TD1 : Manipulation de triangles et objets 3D
- Calcul du centre de gravité et de la normale d'un triangle
- Représentation d'objets 3D par sommets et triangles
- Affichage d'objets polyédriques avec matplotlib

### TD2 : Calcul d'éclairement direct
- Modèle d'éclairement basé sur l'angle normal-source
- Formule : E = max(0, cos(θ))
- Visualisation avec colormap (plt.cm.hot)
- Chargement de modèles STL avec trimesh

### TD3 : Ombres portées
- Algorithme d'intersection rayon-triangle
- Détection des ombres par ray tracing
- Amélioration du réalisme du rendu
- Complexité O(n²) du calcul des ombres

### TD4 : Tessellation adaptative
- Subdivision récursive de triangles
- Fusion d'objets (bâtiment + sol)
- Amélioration de la qualité du rendu par raffinement du maillage
- Seuil de tessellation adaptatif

## Concepts clés de la 3D

- **Maillage triangulaire** : Représentation d'objets 3D par sommets et triangles
- **Normale** : Vecteur perpendiculaire à une surface
- **Ray tracing** : Lancer de rayons pour calculer les intersections
- **Modèle de Lambert** : Éclairement proportionnel à cos(θ)
- **Tessellation** : Subdivision pour augmenter la résolution géométrique

## Pour aller plus loin

### Optimisations possibles :
- **Structures spatiales** (octree, BVH) pour accélérer le ray tracing de O(n²) à O(n log n)
- **Parallélisation** avec multiprocessing pour exploiter plusieurs cœurs CPU
- **Compilation JIT** avec Numba pour accélérer les boucles Python
- **GPU computing** avec CUDA/OpenCL pour le ray tracing massivement parallèle

### Extensions possibles :
- **Sources lumineuses multiples** avec accumulation des éclairements
- **Matériaux différents** (diffus, spéculaire, transparence)
- **Modèle de Phong** pour les reflets spéculaires
- **Réflexions et réfractions** avec ray tracing récursif
- **Textures** pour ajouter des détails visuels
- **Animation** d'objets ou de sources lumineuses
- **Anti-aliasing** pour des rendus plus lisses

### Ressources pour approfondir :
- **Computer Graphics: Principles and Practice** (Foley et al.)
- **Ray Tracing in One Weekend** (Peter Shirley) - gratuit en ligne
- **Physically Based Rendering** (Pharr, Jakob, Humphreys)
- **Scratchapixel** - tutoriels gratuits sur le ray tracing